# Hyperparameter tuning using Grid Search to select the best C & Gamma for SVM classifier

Running this notebook takes too long: https://www.kaggle.com/code/sreedevirajavelu/vreed-modelling-1/edit

This notebook uses a very large range of values for C and gamma using np.logspace(), which creates a logarithmic range of values for each parameter.

By using: 

C_range = np.logspace(-2, 10, 100)

gamma_range = np.logspace(-9, 3, 100)


# Causes of Long Runtime:

Creates a logarithmic space of 100 values for each parameter. This results in a grid search space of 10,000 combinations (100 x 100)

GridSearchCV is initialized with these parameters and a 10-fold cross validation (cv=10)

SVM with an RBF kernel (Radial Basis Function) can be computationally expensive. Each model training involves a solving a quadratic optimization problem, which can be slow.


# Solutions to reduce runtime:

**1. Reduce Parameter Grid:**

Use a coarser grid (eg. 10 values each). Can perform random search or use successive halving to optimize the search process

**2. Use fewer cross-validation folds:**

Reduce cross-validation folds from 10 to 5


**3. Sample the data:**

If dataset is large, work with a subset of the data to perform grid search

**4. Parallelize Grid Search:
**
**GridSearchCV** supports parallel processing with the **n_jobs** **parameter**. 

Setting **n_jobs = -1 ** would utilize all available CPU cores to run multiple cores simultaneously.




In [1]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

#from skl2onnx import convert_sklearn
#from skl2onnx.common.data_types import FloatTensorType, StringTensorType

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Hyperparameter Tuning using Grid Search

To search for the best set of hyperparameters for SVM classifier

In [2]:

C_range = np.logspace(-2, 10, 10)
gamma_range = np.logspace(-9, 3, 10)
param_grid = dict(gamma=gamma_range, C=C_range)
        
eyedata_file_path = '/kaggle/input/vr-eyes-emotions-dataset-vreed/04 Eye Tracking Data/02 Eye Tracking Data (Features Extracted)/EyeTracking_FeaturesExtracted.csv'
eye_data = pd.read_csv(eyedata_file_path) 
eye_data.columns


y = eye_data.Quad_Cat

eye_data_features = ['Num_of_Fixations', 'Mean_Fixation_Duration',
       'SD_Fixation_Duration', 'Skew_Fixation_Duration',
       'Max_Fixation_Duration', 'First_Fixation_Duration', 'Num_of_Saccade',
       'Mean_Saccade_Duration', 'SD_Saccade_Duration', 'Skew_Saccade_Duration',
       'Max_Saccade_Duration', 'Mean_Saccade_Amplitude',
       'SD_Saccade_Amplitude', 'Skew_Saccade_Amplitude',
       'Max_Saccade_Amplitude', 'Mean_Saccade_Direction',
       'SD_Saccade_Direction', 'Skew_Saccade_Direction',
       'Max_Saccade_Direction', 'Mean_Saccade_Length', 'SD_Saccade_Length',
       'Skew_Saccade_Length', 'Max_Saccade_Length', 'Num_of_Blink',
       'Mean_Blink_Duration', 'SD_Blink_Duration', 'Skew_Blink_Duration',
       'Max_Blink_Duration', 'Num_of_Microsac', 'Mean_Microsac_Peak_Vel',
       'SD_Microsac_Peak_Vel', 'Skew_Microsac_Peak_Vel',
       'Max_Microsac_Peak_Vel', 'Mean_Microsac_Ampl', 'SD_Microsac_Ampl',
       'Skew_Microsac_Ampl', 'Max_Microsac_Ampl', 'Mean_Microsac_Dir',
       'SD_Microsac_Dir', 'Skew_Microsac_Dir', 'Max_Microsac_Dir',
       'Mean_Microsac_H_Amp', 'SD_Microsac_H_Amp', 'Skew_Microsac_H_Amp',
       'Max_Microsac_H_Amp', 'Mean_Microsac_V_Amp', 'SD_Microsac_V_Amp',
       'Skew_Microsac_V_Amp', 'Max_Microsac_V_Amp']




# Preprocessing step: Handle missing data

SimpleImputer is used to fill missing values in the feature matrix X.

Replaces any missing values (NaN) with the mean of the respective column

In [3]:
X = eye_data[eye_data_features]

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X)
X_imp = imp.transform(X)


# Perform GridSearchCV with 5 folds of cross validation and find best set of parameters for SVM classifier

In [4]:
gs_knn = GridSearchCV(svm.SVC(), param_grid=param_grid, cv=5) # perform GridSearch with 5 fold cross validation

X_train, X_test, y_train, y_test = train_test_split(X_imp, y, test_size=0.1, random_state=0)

gs_knn.fit(X_train, y_train)
gs_knn.best_params_

#print("Ground Truth: " + str(eye_data.iloc[5].Quad_Cat))
#print("Inference: " + str(test[0]))

# find best model score
gs_knn.score(X_test, y_test)


0.5